# Stellar Velocity

In [19]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic
from binned_stats import get_binned_statistic
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.paths import Paths
from auriga.parser import parse
from auriga.settings import Settings

In [2]:
figure_setup()

In [3]:
def read_data_circ_vel(simulation: str) -> tuple:
    """
    This method returns the radius in the disc plane and the tangential
    velocity of the stars in the main object.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The properties.
    """
    s = Snapshot(simulation=simulation, loadonlytype=[4])
    s.add_extra_coordinates()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    return (s.rho[is_real_star & is_main_obj],
            s.v_phi[is_real_star & is_main_obj],)

## Circular Velocity

In [13]:
def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
    """
    This method adds a panel with the results for a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation.
    ax : plt.Axes
        The ax to which to add the plot.
    """

    rho, v_phi = read_data_circ_vel(simulation=simulation)
    bin_edges = np.linspace(0, 40, 40 + 1)
    mean_v_phi, std, median_v_phi, iqrange, bin_centers = get_binned_statistic(
        x=rho,
        values=v_phi,
        bin_edges=bin_edges,
        xrange=(0, 40)
    )
    ax.fill_between(x=bin_centers,
                    y1=mean_v_phi - std,
                    y2=mean_v_phi + std,
                    color="tab:blue",
                    alpha=0.15,
                    linewidth=0)
    ax.fill_between(x=bin_centers,
                    y1=median_v_phi - iqrange,
                    y2=median_v_phi + iqrange,
                    color="tab:red",
                    alpha=0.15,
                    linewidth=0)
    ax.plot(bin_centers, mean_v_phi, color="tab:blue")
    ax.plot(bin_centers, median_v_phi, color="tab:red")
    ax.text(x=0.05,
            y=0.95,
            s=r"$\texttt{" + simulation.upper() + "}$",
            size=6.0, transform=ax.transAxes,
            ha='left', va='top',
            )

In [5]:
def plot_circ_velocity_for_sample(simulations: list,
                                  filename: str):
    """
    This method creates a plot of the circular velocity as a function of the
    cylindrical radius r_{xy}.

    Parameters
    ----------
    simulations : str
        A list of simulations to plot.
    filename : str
        The name of the output file.
    """

    n_simulations = len(simulations)

    fig = plt.figure(figsize=(7.2, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    set_axs_configuration(
        xlim=(0, 40), ylim=(0, 300),
        xticks=[0, 10, 20, 30], yticks=[0, 100, 200],
        xlabel=r'$r_{xy}$ [ckpc]',
        ylabel=r"$v_\phi$ [$\mathrm{km} \, \mathrm{s}^{-1}$]",
        axs=axs, n_used=n_simulations)

    for idx, ax in enumerate(axs.flat):
        if idx < n_simulations:
            add_panel_for_galaxy(simulations[idx], ax=ax)
        else:
            ax.axis("off")

    for ext in ["pdf", "png"]:
        fig.savefig(
            f"../images/stellar_circ_velocity/circ_velocity_{filename}.{ext}")
    plt.close(fig)

In [15]:
# Create plots for all galaxies in two figures
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
plot_circ_velocity_for_sample(simulations=originals, filename="originals")
plot_circ_velocity_for_sample(simulations=reruns, filename="reruns")

## Velocities by Region

In [16]:
def read_data_vels(simulation: str) -> tuple:
    """
    This method returns the region tag, velocities and radius of the disc plane
    of the stars in the main object of the simulation.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The properties.
    """
    s = Snapshot(simulation=simulation,
                 loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    s.add_reference_to_potential()
    s.add_normalized_potential()
    s.tag_particles_by_region(disc_std_circ=1.0,
                              disc_min_circ=0.4,
                              cold_disc_delta_circ=0.25,
                              bulge_max_specific_energy=-0.75)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    v_z_signed = s.vel[:, 2] * np.sign(s.pos[:, 2])

    return (s.region_tag[is_real_star & is_main_obj],
            s.rho[is_real_star & is_main_obj],
            s.v_rho[is_real_star & is_main_obj],
            s.v_phi[is_real_star & is_main_obj],
            v_z_signed[is_real_star & is_main_obj],
            np.linalg.norm(s.vel, axis=1)[is_real_star & is_main_obj],)

In [24]:
def plot_velocities_by_region(simulation: str):
    """
    This method creates a plot of the velocities as a function of radius in
    the disc plane for each component.

    Parameters
    ----------
    simulation : str
        A list of simulations to plot.
    """

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.grid(True, ls='-', lw=0.25, c='silver', zorder=-20)
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(1, 200)
        ax.set_ylim(-100, 350)
        ax.set_xscale('log')
        ax.set_xticks([1, 10, 100])
        ax.set_yticks([-50, 0, 50, 100, 150, 200, 250, 300])
        ax.set_xticklabels([1, 10, 100])
        ax.set_xlabel(r'$r_{xy}$ [ckpc]')
        ax.set_ylabel(r'Velocity [$\mathrm{km} \, \mathrm{s}^{-1}]$')
        ax.label_outer()

    region_tag, rho, v_rho, v_phi, vz, vel = read_data_vels(
        simulation=simulation)
    for idx, tag in enumerate(np.unique(region_tag)):
        is_region = (region_tag == tag)
        n_bins = np.diff(axs[idx].get_xlim())
        v_rho_bind, bin_edges, _ = binned_statistic(
            x=rho[is_region],
            values=v_rho[is_region],
            bins=n_bins,
            range=axs[idx].get_xlim())
        bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
        axs[idx].plot(bin_centers[np.isfinite(v_rho_bind)],
                      v_rho_bind[np.isfinite(v_rho_bind)],
                      lw=.75, label="$v_r$", zorder=10, c="tab:green")

        v_z_bind, _, _ = binned_statistic(
            x=rho[is_region],
            values=vz[is_region],
            bins=n_bins,
            range=axs[idx].get_xlim())
        axs[idx].plot(bin_centers[np.isfinite(v_z_bind)],
                      v_z_bind[np.isfinite(v_z_bind)],
                      lw=.75, label="$v_z$", zorder=10, c="tab:red")

        v_phi_bind, _, _ = binned_statistic(
            x=rho[is_region],
            values=v_phi[is_region],
            bins=n_bins,
            range=axs[idx].get_xlim())
        axs[idx].plot(bin_centers[np.isfinite(v_phi_bind)],
                      v_phi_bind[np.isfinite(v_phi_bind)],
                      lw=.75, label="$v_\phi$", zorder=10, c="tab:blue")

        v_std_bind, _, _ = binned_statistic(
            x=rho[is_region],
            values=vel[is_region],
            statistic="std",
            bins=n_bins,
            range=axs[idx].get_xlim())
        axs[idx].plot(bin_centers[v_std_bind > 0],
                      v_std_bind[v_std_bind > 0],
                      zorder=10, lw=.75, label="$\sigma_v$", c="black")

        settings = Settings()
        axs[idx].text(x=1.5,
                      y=-80,
                      s=settings.component_labels[settings.components[idx]],
                      size=8.0,
                      ha='left', va='bottom',
                      )

    axs[0].legend(loc="upper left", framealpha=0, fontsize=6.0)

    axs[0].text(x=axs[0].get_xlim()[0],
                y=axs[0].get_ylim()[1],
                s=r"$\texttt{" + simulation.upper() + "}$",
                size=8.0,
                ha='left', va='bottom',
                )

    for ext in ["pdf", "png"]:
        fig.savefig(
            f"../images/stellar_velocities_by_region/"
            f"{simulation}.{ext}")
    plt.close(fig)

In [26]:
# Create plots for all galaxies
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
for simulation in originals + reruns:
    plot_velocities_by_region(simulation=simulation)